In [1]:
import os
import pandas as pd

os. chdir("c:\\Users\\simon\\OneDrive\\Dokumente\\UNILU\\3 - HS24\\1 MA\\business-reports-nlp")

df = pd.read_csv('exploded_texts.csv')
df = df.sample(frac = 1)
df

,file_name,file_path,source,content
17539,capgemini.world_payments_report.2018.txt,reports_digipay_txt/consultancies_pdfminer_cle...,capgemini,7% 7
20050,capgemini_worldpayments_2022.txt,reports_digipay_txt/consultancies_pdfminer_cle...,capgemini,"With APIs, payment firms can embed GTB functi..."
6925,capgemini.world_payments_report.2011.txt,reports_digipay_txt/consultancies_pdfminer_cle...,capgemini,"In the disaggregated value chain, RPSPs would ..."
48053,mckinsey.Lions_go_digital.2013.txt,reports_digipay_txt/consultancies_pdfminer_cle...,mckinsey,strengthening foundations To build a flourish...
40828,mckinsey.Global_flows_in_a_digital_age.2015.txt,reports_digipay_txt/consultancies_pdfminer_cle...,mckinsey,SOURCE: iResearch; Telegeography; OECD; US Bur...
...,...,...,...,...
55511,MGI-Skill-Shift-Automation-and-future-of-the-w...,reports_digipay_txt/consultancies_pdfminer_cle...,mckinsey,"In our survey, companies listed agility and wo..."
44635,mckinsey.Global_payments_A_healthy_industry_co...,reports_digipay_txt/consultancies_pdfminer_cle...,mckinsey,", Traxpay) and alternative finan- technology ..."
25607,World-Payments-Report-WPR-2019.txt,reports_digipay_txt/consultancies_pdfminer_cle...,capgemini,The text of this document was originally writt...
28821,World_Retail_Banking_Report_2021.txt,reports_digipay_txt/consultancies_pdfminer_cle...,capgemini,0 Regulators initiated a new era of open bank...


In [2]:
from sentence_transformers import SentenceTransformer
from umap import UMAP
from hdbscan import HDBSCAN

embedding_model = SentenceTransformer('all-MiniLM-L6-v2')
umap_model = UMAP(n_neighbors=10, n_components=3, min_dist=0.05)
hdbscan_model = HDBSCAN(min_cluster_size=5, min_samples=2,
                        gen_min_span_tree=True,
                        prediction_data=True)

C:\Users\simon\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\umap\distances.py:1063: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
C:\Users\simon\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\umap\distances.py:1071: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecatio

In [3]:
from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import stopwords

stopwords = list(stopwords.words('english'))# + ['http', 'https', 'amp', 'com']

# we add this to remove stopwords that can pollute topcs
vectorizer_model = CountVectorizer(ngram_range=(1, 2), stop_words=stopwords)

In [1]:
from bertopic import BERTopic

model = BERTopic(
    umap_model=umap_model,
    hdbscan_model=hdbscan_model,
    embedding_model=embedding_model,
    vectorizer_model=vectorizer_model,
    top_n_words=5,
    language='english',
    calculate_probabilities=True,
    verbose=True
)
topics, probs = model.fit_transform(df['content'])

KeyboardInterrupt: 

In [28]:
model.get_topic_info()

,Topic,Count,Name,Representation,Representative_Docs
0,0,113,0_payments_banks_payment_services,"[payments, banks, payment, services, digital]",[WORLD PAYMENTS REPORT 2017 TABLE OF CONTE...
1,1,11,1_global_percent_mckinsey_digital,"[global, percent, mckinsey, digital, skills]",[T h e f u t u r e o f w o r k ...


In [29]:
model.visualize_barchart()